In [9]:
import networkx as nx
import csv
from pymongo import MongoClient

In [17]:
gi = nx.read_gexf('data/spanish_ingredients_lexicon_4.gexf')

In [18]:
gt = nx.read_gexf('data/spanish_techniques_lexicon_4.gexf')

In [5]:
i = 'salsa de tomate'
nx.node_connected_component(gi, i)

{'bechamel de tomate',
 'bechamel de tomates',
 'bechamel tomate',
 'bechamel tomates',
 'besamel de tomate',
 'besamel de tomates',
 'besamel tomate',
 'besamel tomates',
 'salsa de tomate',
 'salsa de tomates',
 'salsa tomate',
 'salsa tomates'}

In [6]:
t = 'estofar'
nx.node_connected_component(gt, t)

{'dun',
 'estofado',
 'estofados',
 'estofar',
 'etouffee',
 'etouffée',
 'guisado',
 'guisar',
 'guiso',
 'nimono'}

In [8]:
path_gexf = 'data/elbulli.gexf'
b = nx.read_gexf(path_gexf)

In [11]:
for n in b.nodes_iter(data=True):
    if n[1]['nodetype'] == 'Receta':
        print(n)
        break

('1203', {'nodetype': 'Receta', 'label': '1203', 'titulo': 'moshi: de yogur y frambuesa / de fruta de la pasión y regaliz / de café y guanábana'})


In [17]:
b.node['456']

{'label': '456',
 'nodetype': 'Receta',
 'titulo': 'fruta de la pasión, menta y coco'}

In [18]:
b['456']

{'CÓCTELES': {'edgetype': 'se clasifica', 'id': '15985'},
 'Elaboracion19982001-304': {'edgetype': 'elaboracion', 'id': '15986'},
 'Elaboracion19982001-383': {'edgetype': 'elaboracion', 'id': '15987'},
 'FRÍA': {'edgetype': 'temperatura', 'id': '15984'}}

In [27]:
b['Elaboracion19982001-383']

{'SALADO': {'edgetype': 'mundo', 'id': '11868'},
 'espuma fría': {'edgetype': 'composicion', 'id': '11867'},
 'líquido': {'edgetype': 'composicion', 'id': '11866'},
 'sabor:fruta de la pasión / coco / menta': {'edgetype': 'sabor',
  'id': '11869'}}

In [24]:
for e in b['456']:
    if b['456'][e]['edgetype'] == 'elaboracion':
        print('---')
        print(b.node[e])
        for e2 in b[e]:
            print(e2, b[e][e2])

---
{'nodetype': 'Elaboracion', 'label': 'Elaboracion19982001-304'}
leche de coco {'id': '32643', 'edgetype': 'composicion'}
SALADO {'id': '32642', 'edgetype': 'mundo'}
sabor:coco {'id': '32645', 'edgetype': 'sabor'}
hojas de gelatina {'id': '32644', 'edgetype': 'composicion'}
---
{'nodetype': 'Elaboracion', 'label': 'Elaboracion19982001-383'}
líquido {'id': '11866', 'edgetype': 'composicion'}
sabor:fruta de la pasión / coco / menta {'id': '11869', 'edgetype': 'sabor'}
SALADO {'id': '11868', 'edgetype': 'mundo'}
espuma fría {'id': '11867', 'edgetype': 'composicion'}


In [15]:
for n, m, r in b.edges_iter(data=True):
    print(n, m, r)
    break

Elaboracion20032004-135 infusión de flor de saúco {'id': '15212', 'edgetype': 'composicion'}


In [3]:
apicultur_syns = {}
with open('data/apicultur_techniques_synonyms.csv') as f:
    reader = csv.reader(
        f,
        delimiter=',',
    )
    for row in reader:
        apicultur_syns[row[0]] = row[1:]

In [4]:
def synonyms(x):
    return apicultur_syns.get(x, [])

In [7]:
synonyms('hornear')

['gratinar', 'dorar']

In [13]:
client = MongoClient()
# client.drop_database('recipes')
db = client.recipes

In [19]:
c=0
for row in db.elbulli.find():
    for i in row['ingredients']:
        if i in gi:
            gi.node[i]['count'] += 1
    for t in row['techniques']:
        if t in gt:
            gt.node[t]['count'] += 1
    c+=1
c

1214

In [20]:
nx.write_gexf(gi, 'data/spanish_ingredients_lexicon_5.gexf')

In [21]:
nx.write_gexf(gt, 'data/spanish_techniques_lexicon_5.gexf')

In [22]:
gi = nx.read_gexf('data/spanish_ingredients_lexicon_5.gexf')

In [23]:
gt = nx.read_gexf('data/spanish_techniques_lexicon_5.gexf')

In [37]:
for n in gt.nodes_iter(data=True):
    if n[1]['count'] > 150:
        print(n)

('hervir', {'label': 'hervir', 'count': 390})
('horno', {'label': 'horno', 'count': 294})
('jugo', {'label': 'jugo', 'count': 222})
('sartén', {'label': 'sartén', 'count': 295})
('espuma', {'label': 'espuma', 'count': 169})
('reducir', {'label': 'reducir', 'count': 230})
('agua', {'label': 'agua', 'count': 503})
('sal', {'label': 'sal', 'count': 635})
('secar', {'label': 'secar', 'count': 210})
('min', {'label': 'min', 'count': 553})
('escaldar', {'label': 'escaldar', 'count': 203})
('en agua', {'label': 'en agua', 'count': 438})
('a fuego lento', {'label': 'a fuego lento', 'count': 217})
('puré', {'label': 'puré', 'count': 242})
('saltear', {'label': 'saltear', 'count': 155})
('cocción', {'label': 'cocción', 'count': 167})


In [38]:
elbullig = nx.read_gexf('data/elbulli.gexf')

In [71]:
for n in elbullig.nodes_iter(data=True):
    if n[1]['nodetype'] == 'Receta':
        ingrs = set()
        for e in elbullig[n[0]]:
            if elbullig[n[0]][e]['edgetype'] == 'elaboracion':
                for e2 in elbullig[e]:
                    if elbullig[e][e2]['edgetype'] == 'composicion':
                        ingrs.add(e2)
                    elif elbullig[e][e2]['edgetype'] == 'sabor':
                        ingrs.add(e2[6:])
        print(n[0], ingrs)
        print(db.elbulli.find_one({'_id': n[0]})['ingredients'])
        break

631 {'nuez de macadamia', 'mostaza', 'azúcar', 'azafrán'}
['sal', 'nueces de macadamia', 'mostaza', 'agua', 'azúcar', 'azafrán']


In [46]:
elbullig['631']

{'AMBIENTE': {'edgetype': 'temperatura', 'id': '2518'},
 'Elaboracion19982001-556': {'edgetype': 'elaboracion', 'id': '2519'},
 'SNACKS': {'edgetype': 'se clasifica', 'id': '2520'}}